In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import  mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import r2_score,mean_squared_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")


In [9]:
df = pd.read_csv("data/StudentsPerformance.csv")


In [16]:
x = df.drop('math score',axis = "columns")
y = df['math score']



In [19]:
num_feature = x.select_dtypes(exclude = "object").columns
cat_feature = x.select_dtypes(include = "object").columns
 

In [20]:
from sklearn.preprocessing import  OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
preprocessor = ColumnTransformer([
    ("OneHotEncoder",oh_transformer,cat_feature),
    ("StandardScaler",numeric_transformer,num_feature)
    ])